In [1]:
import random
import json
from pycocotools.coco import COCO
import pandas as pd

In [13]:
# Path to COCO 2017 training annotations file
# Download from: http://images.cocodataset.org/annotations/annotations_trainval2017.zip
ann_file = '/Data2/Arun-UAV/NLP/vision_halu/benchmarks/coco2024/annotations/captions_train2014.json'

# Initialize COCO API for captions
coco = COCO(ann_file)

# Get all image IDs
img_ids = coco.getImgIds()
sample_img_ids = random.sample(img_ids, 20000)

existing_data = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_sample_5000.json")
existing_ids = existing_data["image_id"].unique().tolist()

loading annotations into memory...
Done (t=0.65s)
creating index...
index created!


In [17]:
df = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_batch_1_10000.json")

In [18]:
df

,image_id,file_name,coco_url,captions
0,424083,COCO_train2014_000000424083.jpg,http://images.cocodataset.org/train2014/COCO_t...,[a pizza with some meat and black olives on it...
1,537825,COCO_train2014_000000537825.jpg,http://images.cocodataset.org/train2014/COCO_t...,[A table filled with plates of donuts next to ...
2,197307,COCO_train2014_000000197307.jpg,http://images.cocodataset.org/train2014/COCO_t...,[A woman in a yellow dress holding a tennis ra...
3,167148,COCO_train2014_000000167148.jpg,http://images.cocodataset.org/train2014/COCO_t...,[A sandwich sitting in a basket filled with pa...
4,530936,COCO_train2014_000000530936.jpg,http://images.cocodataset.org/train2014/COCO_t...,[A person jumping up in the air with a snowboa...
...,...,...,...,...
9995,253615,COCO_train2014_000000253615.jpg,http://images.cocodataset.org/train2014/COCO_t...,"[A large herd of sheep in a huge pasture., A v..."
9996,550021,COCO_train2014_000000550021.jpg,http://images.cocodataset.org/train2014/COCO_t...,[A bathroom with a white toilet next to a whit...
9997,118027,COCO_train2014_000000118027.jpg,http://images.cocodataset.org/train2014/COCO_t...,[A kitchen with a white refrigerator and a mic...
9998,138529,COCO_train2014_000000138529.jpg,http://images.cocodataset.org/train2014/COCO_t...,"[A cat sits inside of a bathroom sink., A blac..."


In [14]:
sample_img_ids = [id for id in sample_img_ids if id not in existing_ids]

In [15]:
len(sample_img_ids)

18756

In [16]:


# Randomly sample 5000 image IDs
sample_img_ids = random.sample(sample_img_ids, 10000)

# Load image metadata for the sampled IDs
images = coco.loadImgs(sample_img_ids)

# Collect image paths and captions
samples = []

for img in images:
    img_id = img['id']
    # Each image may have multiple captions; take them all
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    captions = [a['caption'] for a in anns]
    samples.append({
        "image_id": img_id,
        "file_name": img['file_name'],
        "coco_url": img.get('coco_url', None),
        "captions": captions
    })

# Save to JSON file
with open('/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_batch_1_10000.json', 'w', encoding='utf-8') as f:
    json.dump(samples, f, indent=2)

print(f"✅ Saved 5000 sampled images with captions to coco_sample_5000.json")

✅ Saved 5000 sampled images with captions to coco_sample_5000.json


In [32]:
import os
import json
import requests
from tqdm import tqdm

# Output folder
output_dir = '/Data2/Arun-UAV/NLP/vision_halu/train_datasets/poc_5000_coco_images'
os.makedirs(output_dir, exist_ok=True)

# Load sampled JSON
with open('/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_sample_5000.json', 'r') as f:
    samples = json.load(f)

# Download each image
for sample in tqdm(samples, desc="Downloading images"):
    img_url = sample.get('coco_url')
    if not img_url:
        continue
    img_path = os.path.join(output_dir, sample['file_name'])
    if not os.path.exists(img_path):
        try:
            response = requests.get(img_url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(img_path, 'wb') as f_out:
                    for chunk in response.iter_content(1024):
                        f_out.write(chunk)
            else:
                print(f"❌ Failed to download {img_url}")
        except Exception as e:
            print(f"⚠️ Error: {e} for {img_url}")

print("✅ Done! All sampled images downloaded.")


KeyboardInterrupt: 

In [28]:
data = pd.read_json('/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_sample_5000.json')

In [31]:
for row in data.iterrows():
    img_url = row[1]['coco_url']
    break

In [30]:
row

(0,
 image_id                                                 73262
 file_name                      COCO_train2014_000000073262.jpg
 coco_url     http://images.cocodataset.org/train2014/COCO_t...
 captions     [A skateboarder performing a trick next to a b...
 Name: 0, dtype: object)

In [2]:
import os

In [3]:
len(os.listdir("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/poc_5000_coco_images"))

14594

In [5]:
import pandas as pd
df = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_5000_gcp_upload_urs.csv")